In [1]:
import os
import glob
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import datasets
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#### 폴더별로 label이 나누어져 있는 경우
* flow_from_directory()

In [4]:
os.listdir('dataset/mnist_png/training/')

['9', '0', '7', '6', '1', '8', '4', '3', '2', '5']

In [5]:
train_dir = 'dataset/mnist_png/training/'
test_dir = 'dataset/mnist_png/testing/'

### Hyperparameter Tunning

In [15]:
num_epochs = 1
batch_size = 32

learning_rate = 0.001
dropout_rate = 0.7
input_shape = (28, 28, 1)
num_classes = 10

### Preprocess

In [16]:
train_data_gen = ImageDataGenerator(
    rescale=1./255.,
    width_shift_range=0.3,
    zoom_range=0.2,
    horizontal_flip=True
)

test_data_gen = ImageDataGenerator(
    rescale=1./255.,
)

In [17]:
train_data_generator = train_data_gen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    color_mode='grayscale',
)

Found 60000 images belonging to 10 classes.


In [18]:
validation_data_generator = test_data_gen.flow_from_directory(
    test_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical'
)

Found 10000 images belonging to 10 classes.


### Build Model

In [19]:
inputs = layers.Input(input_shape)
l = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
l = layers.Activation('relu')(l)
l = layers.Conv2D(32, (3, 3), padding='SAME')(l)
l = layers.Activation('relu')(l)
l = layers.MaxPool2D(pool_size=(2, 2))(l)
l = layers.Dropout(dropout_rate)(l)

l = layers.Conv2D(64, (3, 3), padding='SAME')(l)
l = layers.Activation('relu')(l)
l = layers.Conv2D(64, (3, 3), padding='SAME')(l)
l = layers.Activation('relu')(l)
l = layers.MaxPool2D(pool_size=(2, 2))(l)
l = layers.Dropout(dropout_rate)(l)

l = layers.Flatten()(l)
l = layers.Dense(512)(l)
l = layers.Activation('relu')(l)
l = layers.Dropout(dropout_rate)(l)
l = layers.Dense(num_classes)(l)
outputs = layers.Activation('softmax')(l)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name='cnn-v1')

In [20]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

### Training

In [21]:
model.fit_generator(train_data_generator,
                   steps_per_epoch=len(train_data_generator),
                   epochs=num_epochs,
                   validation_data=validation_data_generator,
                   validation_steps=len(validation_data_generator))

1875/1875 [==============================] - 145s 78ms/step - loss: 0.8781 - accuracy: 0.6984 - val_loss: 0.4477 - val_accuracy: 0.8700
